In [2]:
import pandas as pd
import numpy as np

from utils.support_functions import haversine

## Data Collection

In [3]:
df_nsw = pd.read_csv('dataset/nsw-property-sales-data-updated20250922.csv')
df_domain = pd.read_csv('dataset/cleaned/domain_realestate_cleaned_all_with_latlong.csv')

In [4]:
df_nsw.head()

,Property ID,Sale counter,Download date / time,Property name,Property unit number,Property house number,Property street name,Property locality,Property post code,Area,Area type,Contract date,Settlement date,Purchase price,Zoning,Nature of property,Primary purpose,Strata lot number,Dealing number,Property legal description
0,3379176.0,1,20240701 01:16,Cams,NaN,154,Sanctuary Rd,Ellalong,2325.0,314100.0,H,2024-06-19,2024-06-25,1800000,RU2,V,Vacant land,NaN,AU186093,1/185087
1,3484622.0,2,20240701 01:16,NaN,NaN,8 A,Redgum Rd,Paxton,2325.0,5280.0,M,2024-05-20,2024-06-27,1150000,R5,R,Residence,NaN,AU194908,31/1144473
2,4140910.0,3,20240701 01:16,NaN,NaN,46,Millbrook Rd,Cliftleigh,2321.0,450.0,M,2024-05-29,2024-06-26,710000,R2,R,Residence,NaN,AU189616,614/1242604
3,1884.0,4,20240701 01:16,NaN,NaN,12,Vincent St,Cessnock,2325.0,1100.0,M,2024-05-14,2024-06-25,1250000,E2,3,Commercial,NaN,AU184079,1/727362
4,4258.0,5,20240701 01:16,NaN,NaN,9,Cessnock Rd,Branxton,2335.0,1113.0,M,2024-04-04,2024-06-27,755000,R3,R,Residence,NaN,AU194943,1/3/8123


In [5]:
df_domain.head()

,Address_URL,Price,Address,Beds,Baths,Parking,Size,Type,format,unit,street_number,street_name,lot_number,suburb,state,postcode,lat,lon
0,https://www.domain.com.au/14-22-26-paul-street...,850000.0,"14/22-26 Paul Street, BALMAIN NSW 2041",1.0,1.0,NaN,NaN,Apartment / Unit / Flat,unit_format,14,22-26,Paul Street,NaN,Balmain,NSW,2041,-33.857815,151.194659
1,https://www.domain.com.au/806-28a-northumberla...,768000.0,"806/28A Northumberland Road, AUBURN NSW 2144",2.0,2.0,1.0,NaN,Apartment / Unit / Flat,unit_format,806,28A,Northumberland Road,NaN,Auburn,NSW,2144,-33.843828,151.037565
2,https://www.domain.com.au/317-318-20-22-davids...,1290000.0,"317-318/20-22 Davidson Street, PORT DOUGLAS Q...",3.0,2.0,1.0,NaN,Apartment / Unit / Flat,unit_format,317-318,20-22,Davidson Street,NaN,Port Douglas,QLD,4877,-16.486914,145.465788
3,https://www.domain.com.au/7-2-crank-street-sun...,1600000.0,"7/2 Crank Street, SUNSHINE BEACH QLD 4567",2.0,1.0,1.0,NaN,Apartment / Unit / Flat,unit_format,7,2,Crank Street,NaN,Sunshine Beach,QLD,4567,-26.406774,153.110846
4,https://www.domain.com.au/3-8-grant-place-port...,369000.0,"3/8 Grant Place, PORT HEDLAND WA 6721",2.0,2.0,NaN,70.0,Apartment / Unit / Flat,unit_format,3,8,Grant Place,NaN,Port Hedland,WA,6721,-20.307055,118.615024


## Data Understanding

In [6]:
print("Domain data shape:", df_domain.shape)
print("NSW Sales data shape:", df_nsw.shape)

Domain data shape: (16864, 18)
NSW Sales data shape: (1827868, 20)


In [7]:
df_domain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16864 entries, 0 to 16863
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Address_URL    16864 non-null  object 
 1   Price          16864 non-null  float64
 2   Address        16864 non-null  object 
 3   Beds           14310 non-null  float64
 4   Baths          14310 non-null  float64
 5   Parking        12645 non-null  float64
 6   Size           8392 non-null   float64
 7   Type           16864 non-null  object 
 8   format         16864 non-null  object 
 9   unit           7901 non-null   object 
 10  street_number  14891 non-null  object 
 11  street_name    16351 non-null  object 
 12  lot_number     1460 non-null   float64
 13  suburb         16864 non-null  object 
 14  state          16864 non-null  object 
 15  postcode       16864 non-null  int64  
 16  lat            16858 non-null  float64
 17  lon            16858 non-null  float64
dtypes: flo

In [8]:
df_nsw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1827868 entries, 0 to 1827867
Data columns (total 20 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Property ID                 float64
 1   Sale counter                int64  
 2   Download date / time        object 
 3   Property name               object 
 4   Property unit number        object 
 5   Property house number       object 
 6   Property street name        object 
 7   Property locality           object 
 8   Property post code          float64
 9   Area                        float64
 10  Area type                   object 
 11  Contract date               object 
 12  Settlement date             object 
 13  Purchase price              int64  
 14  Zoning                      object 
 15  Nature of property          object 
 16  Primary purpose             object 
 17  Strata lot number           float64
 18  Dealing number              object 
 19  Property legal descri

In [9]:
print("Domain missing values:\n", df_domain.isna().sum())

Domain missing values:
 Address_URL          0
Price                0
Address              0
Beds              2554
Baths             2554
Parking           4219
Size              8472
Type                 0
format               0
unit              8963
street_number     1973
street_name        513
lot_number       15404
suburb               0
state                0
postcode             0
lat                  6
lon                  6
dtype: int64


In [10]:
print("NSW missing values:\n", df_nsw.isna().sum())

NSW missing values:
 Property ID                       220
Sale counter                        0
Download date / time                0
Property name                 1747346
Property unit number          1199361
Property house number           39390
Property street name              493
Property locality                   5
Property post code                131
Area                           429118
Area type                      429090
Contract date                     175
Settlement date                     1
Purchase price                      0
Zoning                         762103
Nature of property                  0
Primary purpose                   286
Strata lot number             1141943
Dealing number                      0
Property legal description         19
dtype: int64


## Data Cleaning

### Fill na and drop unused columns

In [11]:
df_domain[['Beds', 'Baths', 'Parking']] = df_domain[['Beds', 'Baths', 'Parking']].fillna(0)
df_nsw[['Zoning', 'Area type']] = df_nsw[['Zoning', 'Area type']].fillna('unknown')

df_nsw['Area'] = df_nsw.groupby('Area type')['Area'].transform(
    lambda x: x.fillna(x.mean())
)

df_nsw['Area'] = df_nsw['Area'].fillna(df_nsw['Area'].mean())

In [12]:
df_nsw = df_nsw.drop(columns=['Property ID', 'Download date / time', "Property legal description", "Settlement date", "Property name", "Strata lot number"])

In [13]:
median_size = df_domain.groupby(['Type', 'Beds'])['Size'].median().reset_index()
median_size = median_size.rename(columns={'Size': 'median_size'})

df_domain = df_domain.merge(median_size, on=['Type', 'Beds'], how='left')

df_domain['Size'] = np.where(df_domain['Size'].isna(), df_domain['median_size'], df_domain['Size'])

df_domain['Size'] = df_domain['Size'].fillna(df_domain['Size'].median())

df_domain = df_domain.drop(columns=['median_size'])

### Drop duplicates

In [14]:
df_domain = df_domain.drop_duplicates()
df_nsw = df_nsw.drop_duplicates()

### Remove records with missing price or postcode

In [15]:
df_domain = df_domain.dropna(subset=["Price", "postcode", "lat", "lon"])
df_nsw = df_nsw.dropna(subset=["Purchase price", "Property post code", "Contract date"])

### Convert prices to numeric

In [16]:
df_domain["Price"] = pd.to_numeric(df_domain["Price"], errors="coerce")
df_nsw["Purchase price"] = pd.to_numeric(df_nsw["Purchase price"], errors="coerce")

### Clean text (strip spaces, lowercase)

In [17]:
for col in ["Address", "street_name", "suburb", "state"]:
    if col in df_domain.columns:
        df_domain[col] = df_domain[col].astype(str).str.strip().str.lower()

for col in ["Property street name", "Property locality"]:
    if col in df_nsw.columns:
        df_nsw[col] = df_nsw[col].astype(str).str.strip().str.lower()

### Street normalization

In [18]:
import re

def normalize_street_name(name: str):
    if not isinstance(name, str):
        return ""
    
    name = name.lower().strip()
    
    replacements = {
        r'\broad\b': 'rd',
        r'\broad\.\b': 'rd',
        r'\bstreet\b': 'st',
        r'\bstreet\.\b': 'st',
        r'\bavenue\b': 'ave',
        r'\bavenue\.\b': 'ave',
        r'\bplace\b': 'pl',
        r'\bdrive\b': 'dr',
        r'\bparade\b': 'pde',
        r'\bhighway\b': 'hwy',
        r'\bcrescent\b': 'cres',
        r'\bclose\b': 'cl',
        r'\blane\b': 'ln',
        r'\bterrace\b': 'tce',
        r'\bboulevard\b': 'blvd'
    }

    for pattern, repl in replacements.items():
        name = re.sub(pattern, repl, name)
    
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()

    return name

df_domain["street_name"] = df_domain["street_name"].apply(normalize_street_name)
df_nsw["Property street name"] = df_nsw["Property street name"].apply(normalize_street_name)

### Feature engineering

In [19]:
SYDNEY = (-33.8688, 151.2093)

df_domain["dist_to_sydney"] = haversine(df_domain["lat"], df_domain["lon"], SYDNEY[0], SYDNEY[1])

In [20]:
df_domain['price_per_m2'] = df_domain['Price'] / df_domain['Size']
df_nsw['price_per_m2'] = df_nsw['Purchase price'] / df_nsw['Area']

### Outlier detection

In [21]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

In [22]:
def detect_outliers(df, features, sample_frac=1.0, random_state=42):
    df = df.copy()

    X = df[features].replace([np.inf, -np.inf], np.nan)
    X = X.fillna(X.median())  
    X = np.clip(X, a_min=-1e9, a_max=1e9)  

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    if sample_frac < 1.0:
        df_sample = df.sample(frac=sample_frac, random_state=random_state)
        X_sample = scaler.transform(df_sample[features].replace([np.inf, -np.inf], np.nan).fillna(X.median()))
    else:
        df_sample = df
        X_sample = X_scaled

    iso = IsolationForest(
        n_estimators=200,
        contamination=0.02,
        random_state=random_state,
        max_samples='auto',
        n_jobs=-1
    )
    iso.fit(X_sample)

    preds = iso.predict(X_scaled)
    df["is_outlier"] = preds == -1

    return df

In [23]:
domain_features = ['Price', 'Beds', 'Baths', 'Parking', 'Size', 'price_per_m2', 'lat', 'lon']
domain_clean = detect_outliers(df_domain, domain_features)
print("Domain outliers detected:", domain_clean["is_outlier"].sum())

Domain outliers detected: 336


In [24]:
nsw_features = ['Purchase price', 'Area', 'price_per_m2']

nsw_clean = detect_outliers(df_nsw, nsw_features, sample_frac=0.4) 
print("NSW outliers detected:", nsw_clean["is_outlier"].sum())


NSW outliers detected: 31673


In [25]:
df_domain = domain_clean[~domain_clean["is_outlier"]].drop(columns="is_outlier")
df_nsw = nsw_clean[~nsw_clean["is_outlier"]].drop(columns="is_outlier")

print(f"Domain cleaned shape: {df_domain.shape}")
print(f"NSW cleaned shape: {df_nsw.shape}")

Domain cleaned shape: (16437, 20)
NSW cleaned shape: (1553001, 15)


## After preprocessing

In [26]:
print("Domain data shape:", df_domain.shape)
print("NSW Sales data shape:", df_nsw.shape)

Domain data shape: (16437, 20)
NSW Sales data shape: (1553001, 15)


In [27]:
df_nsw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1553001 entries, 0 to 1825251
Data columns (total 15 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Sale counter           1553001 non-null  int64  
 1   Property unit number   512046 non-null   object 
 2   Property house number  1521890 non-null  object 
 3   Property street name   1553001 non-null  object 
 4   Property locality      1553001 non-null  object 
 5   Property post code     1553001 non-null  float64
 6   Area                   1553001 non-null  float64
 7   Area type              1553001 non-null  object 
 8   Contract date          1553001 non-null  object 
 9   Purchase price         1553001 non-null  int64  
 10  Zoning                 1553001 non-null  object 
 11  Nature of property     1553001 non-null  object 
 12  Primary purpose        1552774 non-null  object 
 13  Dealing number         1553001 non-null  object 
 14  price_per_m2           

In [28]:
print("Domain missing values:\n", df_domain.isna().sum())

Domain missing values:
 Address_URL           0
Price                 0
Address               0
Beds                  0
Baths                 0
Parking               0
Size                  0
Type                  0
format                0
unit               8696
street_number      1959
street_name           0
lot_number        14986
suburb                0
state                 0
postcode              0
lat                   0
lon                   0
dist_to_sydney        0
price_per_m2          0
dtype: int64


In [29]:
print("NSW missing values:\n", df_nsw.isna().sum())

NSW missing values:
 Sale counter                   0
Property unit number     1040955
Property house number      31111
Property street name           0
Property locality              0
Property post code             0
Area                           0
Area type                      0
Contract date                  0
Purchase price                 0
Zoning                         0
Nature of property             0
Primary purpose              227
Dealing number                 0
price_per_m2                   0
dtype: int64


In [33]:
df_domain_nsw = df_domain[df_domain['state'] == 'nsw']

In [34]:
print("Domain (NSW) missing values:\n", df_domain_nsw.isna().sum())

Domain (NSW) missing values:
 Address_URL          0
Price                0
Address              0
Beds                 0
Baths                0
Parking              0
Size                 0
Type                 0
format               0
unit              3980
street_number      621
street_name          0
lot_number        7554
suburb               0
state                0
postcode             0
lat                  0
lon                  0
dist_to_sydney       0
price_per_m2         0
dtype: int64


In [35]:
df_domain.to_csv('dataset/cleaned/domain.csv', index=False)
df_nsw.to_csv('dataset/cleaned/nsw.csv', index=False)
df_domain_nsw.to_csv('dataset/cleaned/domain_nsw.csv', index=False)